# About

Numerical plots pertaining to trials of an N-dim Kuramoto system with adaptive delays. Requires a folder of .mat files from the the following scripts:
 - scriptND_iter.m
 - script.m

Uses the following packages and functions:

# Library

In [1]:
from __future__ import division, print_function

%matplotlib inline
# Toggle on/off
# %matplotlib notebook

import os
import numpy as np
import scipy.io as sio
from scipy import optimize
from scipy import stats
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.transforms as tsfm
import math
from math import pi

from lib import *

from IPython.display import clear_output

# Directories

In [2]:
# Data files (for fig.4d)
import_folder = 'matlabND_multi_N30_4' # 'matlabND_multi_N30_2'
export_folder = 'export'
ex_name = 'ND_process30_4'

# Directory
dir_main = os.getcwd()
dir_im = os.path.join(dir_main, 'data', import_folder)
dir_ex = os.path.join(dir_main, 'data', export_folder)

filenames = os.listdir(dir_im)

# Import and compile data

## Constant parameters

In [3]:
dir_sam = os.path.join(dir_im, filenames[0])
dict_mat = sio.loadmat(dir_sam)

param = {'g': dict_mat['g'][0,0],
         'omega0': dict_mat['omega0'][0,0],
         'gain': dict_mat['gain'][0,0],
         'tf': dict_mat['tf'][0,0],
         'tau0': dict_mat['tau0'][0,0],
         }

# Asymptotic percentage
asy = 0.1

## Asymptotic values

In [4]:
# Compiled arrays
init_freqs = np.zeros(len(filenames))
init_stds = np.zeros(len(filenames))
asy_freqs = np.zeros(len(filenames))
asy_stds = np.zeros(len(filenames))
var_freqs = np.zeros(len(filenames))

for i in range(len(filenames)):
    
    # Import .mat file
    dir_mat = os.path.join(dir_im, filenames[i])
    dict_mat = sio.loadmat(dir_mat)
    
    # Compile solution dictionary
    sol = {'t': np.reshape(dict_mat['t'], -1),
           'y': dict_mat['y'],
           'yp': dict_mat['yp'],
           'tau': dict_mat['tau'],
           'taup': dict_mat['taup']
          }
    
    init_freq = dict_mat['init_freq'][0,0]
    init_std = dict_mat['std'][0,0]
    
    # Compute the global frequency:
    arr_omega = asylib.weight_avg(sol['t'], sol['yp'], asy)
    asy_Omega = np.sum(arr_omega) / arr_omega.size

    Omega_vars = asylib.weight_avg(sol['t'], (sol['yp'] - asy_Omega)**2, asy)
    Omega_var = np.sum(Omega_vars)

    # Get array of asymptotic phases:
    arr_lin = asy_Omega*sol['t']
    arr_lin = arr_lin[:,None]

    arr_phases = sol['y'] - arr_lin
    asy_phases = asylib.weight_avg(sol['t'], arr_phases, asy)

    # Variance
    phase_vars = asylib.weight_avg(sol['t'], (arr_phases - asy_phases)**2, asy)
    phase_var = np.mean(phase_vars)
    
    N = asy_phases.size
    
    # SAVE TO ARRAYS
    init_freqs[i] = init_freq
    init_stds[i] = init_std
    asy_freqs[i] = asy_Omega
    asy_stds[i] = np.sqrt((N/(N-1))*np.var(asy_phases))
    var_freqs[i] = Omega_var
    
    # Print statements:
    print("Trial: " + filenames[i] + ', asy. freq = ' + str(asy_Omega))

Trial: sol_num0.mat, asy. freq = 0.7799444563552532
Trial: sol_num1.mat, asy. freq = 0.7800207815068169
Trial: sol_num10.mat, asy. freq = 0.780013295750736
Trial: sol_num11.mat, asy. freq = 0.7799616805049621
Trial: sol_num12.mat, asy. freq = 0.7803073892664504
Trial: sol_num13.mat, asy. freq = 0.780027601953117
Trial: sol_num14.mat, asy. freq = 0.7799037136187134
Trial: sol_num15.mat, asy. freq = 0.7799901691642773
Trial: sol_num2.mat, asy. freq = 0.7799505081821316
Trial: sol_num3.mat, asy. freq = 0.7799739426810675
Trial: sol_num4.mat, asy. freq = 0.7800197805399283
Trial: sol_num5.mat, asy. freq = 0.7800381551491552
Trial: sol_num6.mat, asy. freq = 0.7800126486763109
Trial: sol_num7.mat, asy. freq = 0.7800199470201798
Trial: sol_num8.mat, asy. freq = 0.7801708005097951
Trial: sol_num9.mat, asy. freq = 0.7799902820984215


## Export

In [5]:
# Define export dictionary
ex_mat = {'init_freqs': init_freqs,
          'init_stds': init_stds,
          'asy_freqs': asy_freqs,
          'asy_stds': asy_stds,
          'var_freqs': var_freqs,
          'g': param['g'],
          'w0': param['omega0'],
          'tau0': param['tau0'],
          'gain': param['gain']
          }

# Copy parameters
for par in param.keys():
    ex_mat[par] = param[par]

# Export to .mat (export folder)
dir_exfile = os.path.join(dir_ex, ex_name)
sio.savemat(dir_exfile, ex_mat)

# Arrays over time

Skip to this cell if asymptotic arrays have already been processed.

In [6]:
# Import from folder2 with at least two trials (converging to different frequencies
trial1 = {'color': 'blue', 'name': 'sol_num1'}

### Trial

In [7]:
dir_mat1 = os.path.join(dir_im, trial1['name'])
dict_mat1 = sio.loadmat(dir_mat1)

trial1['t'] = np.reshape(dict_mat1['t'], -1)
trial1['y'] = dict_mat1['y']
trial1['yp'] = dict_mat1['yp']
trial1['tau'] = dict_mat1['tau']
trial1['init_freq'] = dict_mat1['init_freq'][0,0]
trial1['std'] = dict_mat1['std'][0,0]

# Compute the asymptotic arrays
arr_omega1 = asylib.weight_avg(trial1['t'], trial1['yp'], asy)
asy_Omega1 = np.sum(arr_omega1) / arr_omega1.size


sol1 = {'t': np.reshape(dict_mat1['t'], -1),
       'y': dict_mat1['y'],
       'yp': dict_mat1['yp'],
       'tau': dict_mat1['tau'],
       'taup': dict_mat1['taup']
      }

# Get array of asymptotic phases:
arr_lin1 = asy_Omega1*sol1['t']
arr_lin1 = arr_lin1[:,None]

arr_phases1 = trial1['y'] - arr_lin1
asy_phases1 = asylib.weight_avg(trial1['t'], arr_phases1, asy)

# Mean zero asy phases:
cen_phases = asy_phases1 - np.mean(asy_phases1)

# Store
trial1['phases'] = arr_phases1
trial1['asy_phases'] = asy_phases1
trial1['asy_Omega'] = asy_Omega1

# Asymptotic arrays

In [8]:
# Import arrays and values
dir_exfile = os.path.join(dir_ex, ex_name)
dict_ex = sio.loadmat(dir_exfile)

init_freqs = np.reshape(dict_ex['init_freqs'], -1)
init_stds = np.reshape(dict_ex['init_stds'], -1)
asy_freqs = np.reshape(dict_ex['asy_freqs'], -1)
asy_stds = np.reshape(dict_ex['asy_stds'], -1)

# Theoretical arrays

## Stability states curve

In [9]:
# Parameters
g = dict_ex['g'][0,0]
w0 = dict_ex['w0'][0,0]
tau0 = dict_ex['tau0'][0,0]
gain = dict_ex['gain'][0,0]

param = {'g': g,
         'omega0': w0,
         'tau0': tau0, 
         'gain': gain
         }

steps = 400

# Fixed-point functions
Omega_fun = lambda Omega, delta2: funlib.Omega_infty(Omega, delta2, param, L=2*pi, steps=steps) - Omega

# Generate error meshes of Omega_fun
part_Omega = 100
part_delta = 100

cap = 0.5

# Processing function
z_fun = lambda x: x

# Generate error meshes
Omega_arr = np.linspace(w0 - g, w0 + g, num=part_Omega)
delta_arr = np.linspace(0,1, num=part_delta)
delta_mesh, Omega_mesh = np.meshgrid(delta_arr, Omega_arr)

err_mesh = np.zeros(Omega_mesh.shape)
for j in range(Omega_arr.size):
    for k in range(delta_arr.size):
        new_err = Omega_fun(Omega_mesh[j,k], delta_mesh[j,k]**2)
        err_mesh[j,k] = np.max((np.min((new_err, cap)), -cap))

# Obtain solution curve Omega = Omega(delta)
Omega_roots = np.zeros(delta_arr.size+1)
Omega_roots[0] = w0

for k in range(delta_arr.size):
    deltak = delta_arr[k]
    Omega_funk = lambda Omega: Omega_fun(Omega, deltak**2)
    new_root = rootlib.find_root(Omega_funk, Omega_roots[k], w0-g, w0+g, steps=100)
    if new_root[1]:
        Omega_roots[k+1] = new_root[0]
        
Omega_roots = Omega_roots[1:]

## Stable synchronization states through eigenvalue analysis

In [10]:
# Numerical options
steps = 1000
L = 2*pi
deg = 3
R = 10

# Error function
err_fun = lambda z: eigenlib.eig_infty_scale(z, Omega, delta2, param, deg=deg, R=R, L=L, steps=steps) - z
eig_terms = lambda u,v2: eigenlib.eig_infty_terms(deg, u, v2, param, R=1, L=pi, steps=steps)

delta_arr0 = delta_arr[1:]
Omega_roots0 = Omega_roots[1:]

real_max = np.zeros(delta_arr0.size)
inds_stable = []

for k in range(delta_arr0.size):
    delta2 = delta_arr0[k]**2
    Omega = Omega_roots0[k]
    
    new_eigterms = eig_terms(Omega, delta2)
    coeffs = new_eigterms[0]
    coeffs[1] += -1
    coeffs[0] += -new_eigterms[2]
    
    # Filter out any roots close to 0
    eig_roots = np.roots(np.flip(coeffs,0))
    eig_roots = eig_roots[np.abs(eig_roots) > 1e-14]
    
    max_real = np.max(np.real(eig_roots))
    real_max[k] = np.min((max_real,1))
    
    if real_max[k] <= 0:
        inds_stable.append(k)

inds_stable = np.array(inds_stable)
delta_stable = delta_arr0[inds_stable]
Omega_stable = Omega_roots0[inds_stable]

# Plot

## Options

In [11]:
# Plot positions [left, bottom, width, height] 
bbox0 = [0.15, 0.7, 0.35, 0.20]  # Fig 2a: Derivative plots
bbox1 = [0.15, 0.4, 0.35, 0.20]  # Fig 2b: sin(phase) plots
bbox2 = [0.15, 0.1, 0.35, 0.20] # Fig 2c: Delay plots
bbox3 = [0.60, 0.1, 0.45, 0.5] # Fig 2d: Iterated trial plot
bbox4 = [0.60, 0.7, 0.45, 0.2] # Fig 2d: Iterated trial plot

# Figure a,b,c lines
line_opts = {'linewidth': 1.2,
             'alpha': 0.4
            }

tf = np.max(trial1['t'])

# Dotted lines (linking initial to asy points in fig.d)
dotopts = {'linestyle': '-',
           'linewidth': 0.5,
           'marker': None,
           'color': 'black',
           'alpha': 0.5
          }

# Sample which delay indices to plot:
NN = trial1['tau'].shape[1]
num_inds = 50
inds = np.random.choice(np.arange(NN), size=num_inds, replace=False)

# Histogram
L_dist = 5*np.sqrt(np.var(cen_phases))
bins_dist = np.linspace(-L_dist, L_dist, num=30)

## Generate figure

Shapiro-Wilk Test p-value: 0.054289333522319794
{'g': 1.5, 'omega0': 1, 'tau0': 0.1, 'gain': 60}


In [12]:
fig, ax = plt.subplots(5, figsize=(12,10), dpi=80)

# Set positions
ax[0].set_position(bbox0)
ax[1].set_position(bbox1)
ax[2].set_position(bbox2)
ax[3].set_position(bbox3)
ax[4].set_position(bbox4)

# Trial 1:
ax[0].plot(trial1['t'], trial1['yp'], **line_opts)
ax[1].plot(trial1['t'], np.sin(trial1['phases']), **line_opts)
ax[2].plot(trial1['t'], trial1['tau'][:,inds], **line_opts)

ax[0].axhline(y=dict_ex['w0'], color='green', alpha=0.8, linewidth=1.5)

for k in range(init_freqs.size):
    p1 = np.array([init_freqs[k], asy_freqs[k]])
    p2 = np.array([init_stds[k], asy_stds[k]])
    ax[3].plot(p1, p2, **dotopts, zorder=-2)

ax[3].plot(Omega_roots, delta_arr, color='blue', alpha=0.4, zorder=-4)
ax[3].plot(Omega_stable, delta_stable, color='green', linewidth=3.0, zorder=-3)
ax[3].scatter(init_freqs, init_stds, color='orange', zorder=-1)
ax[3].scatter(asy_freqs, asy_stds, color='magenta', zorder=1)

ax[3].axvline(x=dict_ex['w0'], color='green', alpha=0.5, zorder=-2)
ax[3].axhline(y=0, color='black', alpha=0.5, linewidth=0.5, zorder=-2)


ax[4].hist(cen_phases, bins=bins_dist)

# Adjustments to axes
ax[0].set_xlim(left=np.min(trial1['t']), right=tf)
ax[1].set_xlim(left=np.min(trial1['t']), right=tf)
ax[2].set_xlim(left=np.min(trial1['t']), right=tf)

print("Shapiro-Wilk Test p-value: " + str(stats.shapiro(cen_phases)[1])) # High p-value implies normal distribution
print(param)

In [13]:
print(trial1['color'] + ' freq: ' + str(trial1['asy_Omega']) + ', initial phase std: ' + str(trial1['std']))

blue freq: 0.7800207815068169, initial phase std: 0.6027014802313547


In [14]:
Omega_fun(asy_freqs[0], asy_stds[0]**2), asy_stds[1], asy_freqs[1]

(-0.07736730908764156, 0.06049190097672986, 0.7800207815068169)

In [15]:
asy_stds

array([0.06053506, 0.0604919 , 0.06050787, 0.06052686, 0.0604603 ,
       0.06052308, 0.06085514, 0.06051981, 0.06056835, 0.06053027,
       0.06050402, 0.06051573, 0.06051628, 0.06047634, 0.0605466 ,
       0.06054501])